# Minprj : 
>date : 2 nov 2021<br>
>author : bae hueng myoung
>e-mail : naroo135@gmail.com

## work(0) : data load 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/MachineLearning/Aiffel_LMS/EXP09_recommendation')
file_dir = '/content/drive/MyDrive/MachineLearning/Aiffel_LMS/EXP09_recommendation'

In [3]:
import pandas as pd
import os
rating_file_path= file_dir + '/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings','timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


## work(1) : preprocessing

###  필요없는 TIMESTAMP 정보를 제거하였다.

In [4]:
movie_ratings = ratings[['user_id','movie_id','ratings']]
movie_ratings

,user_id,movie_id,ratings
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
1000204,6040,1091,1
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4


In [5]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [27]:
result = ratings.groupby('user_id')['movie_id'].count()
result.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

#### 유저별로 영화를 얼마나 보았는지 평균, 분산, 그리고 상위 75% 50% 25%의 영화 시청 수를 확인할 수 있다. 

In [6]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [7]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [8]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path= file_dir + '/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


## work(2) : add my favorite movies



In [9]:
print('unique user num : ',ratings['user_id'].nunique())
print('uniuqe movies num : ',movies['movie_id'].nunique())


unique user num :  6039
uniuqe movies num :  3883


In [10]:
movies_count = ratings.groupby('movie_id')['user_id'].count()
movies_thirty = movies_count.sort_values(ascending=False).head(30)
movies_thirty


movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

In [11]:
top30_list = []
for idx,cnt in movies_thirty.items():
    top30_list.append(idx)

for idx in top30_list:
    print(movies[movies['movie_id'] == idx])

      movie_id                   title         genre
2789      2858  American Beauty (1999)  Comedy|Drama
     movie_id  ...                            genre
257       260  ...  Action|Adventure|Fantasy|Sci-Fi

[1 rows x 3 columns]
      movie_id  ...                              genre
1178      1196  ...  Action|Adventure|Drama|Sci-Fi|War

[1 rows x 3 columns]
      movie_id  ...                                genre
1192      1210  ...  Action|Adventure|Romance|Sci-Fi|War

[1 rows x 3 columns]
      movie_id                       title             genre
1959      2028  Saving Private Ryan (1998)  Action|Drama|War
     movie_id                              title                   genre
585       589  Terminator 2: Judgment Day (1991)  Action|Sci-Fi|Thriller
     movie_id                             title           genre
589       593  Silence of the Lambs, The (1991)  Drama|Thriller
      movie_id                           title             genre
1180      1198  Raiders of the Lost Ark

#### 데이터셋에서 별점3점 이상의 작품에서 가장 많이 본 작품들 30개를 출력

In [12]:
movie_ratings['movie_id'].sort_values()

427702       1
1966         1
683688       1
596207       1
465902       1
          ... 
84701     3952
253845    3952
180689    3952
35180     3952
372755    3952
Name: movie_id, Length: 1000209, dtype: int64

In [13]:
movie_ratings['user_id'].unique()

array([   1,    2,    3, ..., 6038, 6039, 6040])

#### movie id가 3952까지 있으므로 나의 추가할 페이보릿 무비 아이디는 2953부터 ASSIGN한다.
user id는 6040까지 이므로 신규 user id는 6041로 추가한다.

In [14]:
my_favorite_movies = ['ironman' , 'squid  game' ,'matrix' ,'source code' ,'black hawk down']
my_favorite_movies_id = [3953,3954,3955,3956,3957]

new_id = {'benjamin' : 6041}
my_favor_ratings = pd.DataFrame({'user_id': [new_id['benjamin']]*5, 'movie_id': my_favorite_movies_id, 'ratings':[5,4,4,5,5]})
my_favor_ratings


,user_id,movie_id,ratings
0,6041,3953,5
1,6041,3954,4
2,6041,3955,4
3,6041,3956,5
4,6041,3957,5


In [15]:
if not movie_ratings.isin({'user_id':[new_id['benjamin']]})['user_id'].any():  
    movie_ratings = movie_ratings.append(my_favor_ratings)                         

movie_ratings.tail(10) 

,user_id,movie_id,ratings
1000204,6040,1091,1
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4
1000208,6040,1097,4
0,6041,3953,5
1,6041,3954,4
2,6041,3955,4
3,6041,3956,5
4,6041,3957,5


#### benjamin(user id = 6041)의 이름으로 5개의 무비아이디가 추가된 것을 볼 수 있다.

## work(3) : CSR matrix

In [16]:
movie_ratings.user_id


0       1
1       1
2       1
3       1
4       1
     ... 
0    6041
1    6041
2    6041
3    6041
4    6041
Name: user_id, Length: 1000214, dtype: int64

In [18]:
from scipy.sparse import csr_matrix

num_user = movie_ratings['user_id'].nunique()
num_movie = movie_ratings['movie_id'].nunique()

csr_data = csr_matrix((movie_ratings.ratings, (movie_ratings.user_id, movie_ratings.movie_id)), shape=(num_user, num_movie))
csr_data

ValueError: ignored

## work (4) : als modeling
colab에서 implicit 패키지를 이용하기 위해선 아래와 같이 패키지를 설치하여야 한다.
ref : https://colab.research.google.com/github/denisparra/pyreclab_tutorial/blob/master/implicit_als_vs_bpr.ipynb

In [ ]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [ ]:
!curl -L -o "u2.base" "https://drive.google.com/uc?export=download&id=1bGweNw7NbOHoJz11v6ld7ymLR8MLvBsA"
!curl -L -o "u2.test" "https://drive.google.com/uc?export=download&id=1f_HwJWC_1HFzgAjKAWKwkuxgjkhkXrVg"
!curl -L -o "u.item" "https://drive.google.com/uc?export=download&id=10YLhxkO2-M_flQtyo9OYV4nT9IvSESuz"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    501      0 --:--:-- --:--:-- --:--:--   501
100 1546k  100 1546k    0     0  1311k      0  0:00:01  0:00:01 --:--:-- 1311k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    657      0 --:--:-- --:--:-- --:--:--   656
100  385k  100  385k    0     0   450k      0 --:--:-- --:--:-- --:--:--  450k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    739      0 --:--:-- --:--:-- --:--:--   737
100  230k  100  230k    0     0   304k      0 --:--:-- --:--:-- --:--:--  304k


In [ ]:
!pip3 install pandas --upgrade
!pip3 install implicit --upgrade

In [ ]:
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [ ]:
csr_data_transpose = csr_data.T
csr_data_transpose

In [ ]:
zimin, black_eyed_peas = user_to_idx['zimin'], artist_to_idx['black eyed peas']
zimin_vector, black_eyed_peas_vector = als_model.user_factors[zimin], als_model.item_factors[black_eyed_peas]
